# utility.parser

In [1]:
# 命令行参数
import argparse

def parse_args():
    parser = argparse.ArgumentParser(description="Run NGCF.")
    parser.add_argument('--weights_path', nargs='?', default='',
                        help='Store model path.')
    parser.add_argument('--data_path', nargs='?', default='Data/',
                        help='Input data path.')
    parser.add_argument('--proj_path', nargs='?', default='',
                        help='Project path.')

    parser.add_argument('--dataset', nargs='?', default='gowalla',
                        help='Choose a dataset from {gowalla, yelp2018, amazon-book}')
    parser.add_argument('--pretrain', type=int, default=0,
                        help='0: No pretrain, -1: Pretrain with the learned embeddings, 1:Pretrain with stored models.')
    parser.add_argument('--verbose', type=int, default=1,
                        help='Interval of evaluation.')
    parser.add_argument('--is_norm', type=int, default=1,
                    help='Interval of evaluation.')
    parser.add_argument('--epoch', type=int, default=1000,
                        help='Number of epoch.')

    parser.add_argument('--embed_size', type=int, default=64,
                        help='Embedding size.')
    parser.add_argument('--layer_size', nargs='?', default='[64, 64, 64, 64]',
                        help='Output sizes of every layer')
    parser.add_argument('--batch_size', type=int, default=1024,
                        help='Batch size.')

    parser.add_argument('--regs', nargs='?', default='[1e-5,1e-5,1e-2]',
                        help='Regularizations.')
    parser.add_argument('--lr', type=float, default=0.01,
                        help='Learning rate.')

    parser.add_argument('--model_type', nargs='?', default='lightgcn',
                        help='Specify the name of model (lightgcn).')
    parser.add_argument('--adj_type', nargs='?', default='pre',
                        help='Specify the type of the adjacency (laplacian) matrix from {plain, norm, mean}.')
    parser.add_argument('--alg_type', nargs='?', default='lightgcn',
                        help='Specify the type of the graph convolutional layer from {ngcf, gcn, gcmc}.')

    parser.add_argument('--gpu_id', type=int, default=0,
                        help='0 for NAIS_prod, 1 for NAIS_concat')

    parser.add_argument('--node_dropout_flag', type=int, default=0,
                        help='0: Disable node dropout, 1: Activate node dropout')
    parser.add_argument('--node_dropout', nargs='?', default='[0.1]',
                        help='Keep probability w.r.t. node dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')
    parser.add_argument('--mess_dropout', nargs='?', default='[0.1]',
                        help='Keep probability w.r.t. message dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')

    parser.add_argument('--Ks', nargs='?', default='[20]',
                        help='Top k(s) recommend')

    parser.add_argument('--save_flag', type=int, default=0,
                        help='0: Disable model saver, 1: Activate model saver')

    parser.add_argument('--test_flag', nargs='?', default='part',
                        help='Specify the test type from {part, full}, indicating whether the reference is done in mini-batch')

    parser.add_argument('--report', type=int, default=0,
                        help='0: Disable performance report w.r.t. sparsity levels, 1: Show performance report w.r.t. sparsity levels')

    # JpyNotebook 上报错，Pycharm上不会报错，必须传入args参数
    return parser.parse_args(args=[])

# utility.batch_test

# 使用的数据集

## 可以在命令行中切换 gowalla，yelp2018，amazon-book 三个数据集，默认使用gowalla数据集
## gowalla：美国微博好友关系数据集，每一条记录表示两两对应的朋友关系
## yelp2018：美国商户店铺打分数据集，每一条包含用户对哪些商铺打了好评
## amazon-book：亚马逊书籍评论数据集，每一条记录用户喜欢哪些书

# ·训练集：每一条代表某个用户对哪些商品更青睐
# ·测试集：与训练集类似，不过其中未出现的商品会被认为用户讨厌这件商品

In [2]:
# from utility.parser import parse_args
from utility.load_data import *
from evaluator import eval_score_matrix_foldout
import multiprocessing
import heapq
import numpy as np
cores = multiprocessing.cpu_count() // 2

# ************************
args = parse_args()

# 创建数据集，默认数据集为gowalla
data_generator = Data(path=args.data_path + args.dataset, batch_size=args.batch_size)
USR_NUM, ITEM_NUM = data_generator.n_users, data_generator.n_items
N_TRAIN, N_TEST = data_generator.n_train, data_generator.n_test

BATCH_SIZE = args.batch_size


# 给定test函数，用于测试推荐系统模型在给定用户集上的性能表现
"""
    sess：TensorFlow 会话对象；
    model：推荐系统模型对象；
    users_to_test：待测试的用户 ID 列表；
    drop_flag：是否启用 Dropout，默认为 False；
    train_set_flag：是否使用训练集作为测试集，默认为 0。
"""
def test(sess, model, users_to_test, drop_flag=False, train_set_flag=0):
    # B: batch size
    # N: the number of items
    # top_show，表示要展示的，待排序的推荐列表
    top_show = np.sort(model.Ks)
    max_top = max(top_show)
    # 将初值全部设为0，方便在测试中累加推荐列表
    result = {'precision': np.zeros(len(model.Ks)), 'recall': np.zeros(len(model.Ks)), 'ndcg': np.zeros(len(model.Ks))}
    
    u_batch_size = BATCH_SIZE
    
    # 获取测试用户数量并分入若干个batch中
    test_users = users_to_test
    n_test_users = len(test_users)
    n_user_batchs = n_test_users // u_batch_size + 1
    
    count = 0
    all_result = []
    # 获取商品列表
    item_batch = range(ITEM_NUM)
    # 遍历每个用户批次
    for u_batch_id in range(n_user_batchs):
        # 设置批次起点与终点
        start = u_batch_id * u_batch_size
        end = (u_batch_id + 1) * u_batch_size
        user_batch = test_users[start: end]
        # 获取batch内的用户对所有物品的评分（预测数据）
        if drop_flag == False:
            rate_batch = sess.run(model.batch_ratings, {model.users: user_batch,
                                                        model.pos_items: item_batch})
        else:
            rate_batch = sess.run(model.batch_ratings, {model.users: user_batch,
                                                        model.pos_items: item_batch,
                                                        # 以0.1，0.2，0.3...的dropout率变化（三层）
                                                        model.node_dropout: [0.] * len(eval(args.layer_size)),
                                                        model.mess_dropout: [0.] * len(eval(args.layer_size))})
        rate_batch = np.array(rate_batch)# (B, N)
        # 真实数据
        test_items = []
        # 如果使用的是测试集
        if train_set_flag == 0:
            # 遍历batch中的每个用户
            for user in user_batch:
                # 获取与用户有交互的商品id
                test_items.append(data_generator.test_set[user])# (B, #test_items)
                
            """
            训练集和测试集是分开的，训练集包含了用户已经交互过的商品，而测试集包含了用户未曾交互过的商品。
            在测试阶段，我们需要给用户推荐他们未曾交互过的商品，因此需要将训练集中的商品排除在推荐列表之外。
            因此可以将训练集中的商品的评分设置为负无穷，这样它们就会被排在推荐列表的末尾。
            如果使用训练集就不需要这一步。
            """
            for idx, user in enumerate(user_batch):
                    train_items_off = data_generator.train_items[user]
                    rate_batch[idx][train_items_off] = -np.inf
        # 如果使用的是训练集
        else:
            for user in user_batch:
                test_items.append(data_generator.train_items[user])
        # 利用评分矩阵获取最终结果，(B,k*metric_num), max_top= 20
        batch_result = eval_score_matrix_foldout(rate_batch, test_items, max_top)
        # 记录下本次处理了几个用户评分
        count += len(batch_result)
        all_result.append(batch_result)
        
    # 检测是否所有用户都完成打分
    assert count == n_test_users
    # 拼装所有批次的评估结果
    all_result = np.concatenate(all_result, axis=0)
    # 计算所有评估结果的平均值
    final_result = np.mean(all_result, axis=0)  # mean
    final_result = np.reshape(final_result, newshape=[5, max_top])
    final_result = final_result[:, top_show-1]
    final_result = np.reshape(final_result, newshape=[5, len(top_show)])
    result['precision'] += final_result[0]
    result['recall'] += final_result[1]
    result['ndcg'] += final_result[3]
    return result

eval_score_matrix_foldout with python
n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084


In [ ]:
# 上一步中使用的eval_score_matrix_foldout函数
def eval_score_matrix_foldout(score_matrix, test_items, top_k=50, thread_num=None):
    # 针对每个测试用户计算一组评估指标
    def _eval_one_user(idx):
        # 该用户的所有打分
        scores = score_matrix[idx]  
        # 该用户打分的所有物品
        test_item = test_items[idx]  
        
        # 返回前K的评分最高的物品
        ranking = argmax_top_k(scores, top_k)  # Top-K items
        result = []
        result.extend(precision(ranking, test_item)) # 精准率
        result.extend(recall(ranking, test_item))    # 召回率
        result.extend(map(ranking, test_item))       # 平均精确率
        result.extend(ndcg(ranking, test_item))      # 归一化折损累计增益，评价排序效果
        result.extend(mrr(ranking, test_item))       # 平均倒数排名，评价排序效果

        result = np.array(result, dtype=np.float32).flatten()
        return result
    
    # 多线程运算
    with ThreadPoolExecutor(max_workers=thread_num) as executor:
        batch_result = executor.map(_eval_one_user, range(len(test_items)))
    
    # 返回一个包含了所有测试用户评估结果的数组
    result = list(batch_result)  # generator to list
    return np.array(result)  # list to ndarray

# 使用到的函数
"""
def argmax_top_k(a, top_k=50):
    ele_idx = heapq.nlargest(top_k, zip(a, itertools.count()))
    return np.array([idx for ele, idx in ele_idx], dtype=np.intc)

def precision(rank, ground_truth):
    hits = [1 if item in ground_truth else 0 for item in rank]
    result = np.cumsum(hits, dtype=np.float)/np.arange(1, len(rank)+1)
    return result


def recall(rank, ground_truth):
    hits = [1 if item in ground_truth else 0 for item in rank]
    result = np.cumsum(hits, dtype=np.float) / len(ground_truth)
    return result


def map(rank, ground_truth):
    pre = precision(rank, ground_truth)
    pre = [pre[idx] if item in ground_truth else 0 for idx, item in enumerate(rank)]
    sum_pre = np.cumsum(pre, dtype=np.float32)
    gt_len = len(ground_truth)
    # len_rank = np.array([min(i, gt_len) for i in range(1, len(rank)+1)])
    result = sum_pre/gt_len
    return result


def ndcg(rank, ground_truth):
    len_rank = len(rank)
    len_gt = len(ground_truth)
    idcg_len = min(len_gt, len_rank)

    # calculate idcg
    idcg = np.cumsum(1.0 / np.log2(np.arange(2, len_rank + 2)))
    idcg[idcg_len:] = idcg[idcg_len-1]

    # idcg = np.cumsum(1.0/np.log2(np.arange(2, len_rank+2)))
    dcg = np.cumsum([1.0/np.log2(idx+2) if item in ground_truth else 0.0 for idx, item in enumerate(rank)])
    result = dcg/idcg
    return result


def mrr(rank, ground_truth):
    last_idx = sys.maxsize
    for idx, item in enumerate(rank):
        if item in ground_truth:
            last_idx = idx
            break
    result = np.zeros(len(rank), dtype=np.float32)
    result[last_idx:] = 1.0/(last_idx+1)
    return result
"""

In [3]:
import os
import sys
import threading
# Tensorflow版本为2.11.0，需要降到实验环境的1.11.0
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# import tensorflow as tf

from tensorflow.python.client import device_lib
from utility.helper import *
# from utility.batch_test import *
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

cpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'CPU']

Instructions for updating:
non-resource variables are not supported in the long term


In [1]:
class LightGCN(object):
    def __init__(self, data_config, pretrain_data):
        # 参数定义
        self.model_type = 'LightGCN'# 模型类型
        self.adj_type = args.adj_type # 邻接矩阵类型
        self.alg_type = args.alg_type # 算法类型
        self.pretrain_data = pretrain_data # 预处理数据（可选择加载）
        self.n_users = data_config['n_users'] # 用户数量
        self.n_items = data_config['n_items'] # 商品数量
        self.n_fold = 100 # 数据集折数
        self.norm_adj = data_config['norm_adj'] # 乘以度矩阵之后的邻接矩阵
        self.n_nonzero_elems = self.norm_adj.count_nonzero() # norm_adj中非零元的数量
        self.lr = args.lr # 学习率
        self.emb_dim = args.embed_size # 嵌入维度
        self.batch_size = args.batch_size # 批大小
        self.weight_size = eval(args.layer_size) # 隐层权重尺寸
        self.n_layers = len(self.weight_size) # 隐层数量
        self.regs = eval(args.regs) # 正则化参数
        self.decay = self.regs[0]   # L2正则化系数
        self.log_dir=self.create_model_str() # 日志目录
        self.verbose = args.verbose # 详细信息
        self.Ks = eval(args.Ks) # 推荐商品列表


        '''
        *********************************************************
        Create Placeholder for Input Data & Dropout.
        '''
        # 定义tensorflow中的占位符
        self.users = tf.placeholder(tf.int32, shape=(None,)) 
        self.pos_items = tf.placeholder(tf.int32, shape=(None,)) # 正物品
        self.neg_items = tf.placeholder(tf.int32, shape=(None,)) # 负物品
        
        self.node_dropout_flag = args.node_dropout_flag
        self.node_dropout = tf.placeholder(tf.float32, shape=[None]) # 节点和消息权重的dropout
        self.mess_dropout = tf.placeholder(tf.float32, shape=[None])

        with tf.name_scope('TRAIN_LOSS'):
            """
            self.train_loss：训练总损失
            self.train_mf_loss：损失函数
            self.train_emb_loss：正则化项
            self.train_reg_loss：常数0，可以和其他损失函数拼接，防止维度不匹配
            """
            self.train_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('train_loss', self.train_loss)
            self.train_mf_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('train_mf_loss', self.train_mf_loss)
            self.train_emb_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('train_emb_loss', self.train_emb_loss)
            self.train_reg_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('train_reg_loss', self.train_reg_loss)
        # 张量合并
        self.merged_train_loss = tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES, 'TRAIN_LOSS'))
        
        
        with tf.name_scope('TRAIN_ACC'):
            """
            self.train_rec_first：最推荐位置的召回率
            self.train_rec_last：最不推荐位置的召回率
            """
            self.train_rec_first = tf.placeholder(tf.float32)
            #record for top(Ks[0])
            tf.summary.scalar('train_rec_first', self.train_rec_first)
            self.train_rec_last = tf.placeholder(tf.float32)
            #record for top(Ks[-1])
            tf.summary.scalar('train_rec_last', self.train_rec_last)
            self.train_ndcg_first = tf.placeholder(tf.float32)
            tf.summary.scalar('train_ndcg_first', self.train_ndcg_first)
            self.train_ndcg_last = tf.placeholder(tf.float32)
            tf.summary.scalar('train_ndcg_last', self.train_ndcg_last)
        self.merged_train_acc = tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES, 'TRAIN_ACC'))

        with tf.name_scope('TEST_LOSS'):
            """
            self.test_loss：测试总损失
            self.test_mf_loss：测试MF损失
            self.test_emb_loss：测试嵌入损失
            self.test_reg_loss：测试正则化损失
            
            """
            self.test_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('test_loss', self.test_loss)
            self.test_mf_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('test_mf_loss', self.test_mf_loss)
            self.test_emb_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('test_emb_loss', self.test_emb_loss)
            self.test_reg_loss = tf.placeholder(tf.float32)
            tf.summary.scalar('test_reg_loss', self.test_reg_loss)
        self.merged_test_loss = tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES, 'TEST_LOSS'))

        with tf.name_scope('TEST_ACC'):
            # 同理
            self.test_rec_first = tf.placeholder(tf.float32)
            tf.summary.scalar('test_rec_first', self.test_rec_first)
            self.test_rec_last = tf.placeholder(tf.float32)
            tf.summary.scalar('test_rec_last', self.test_rec_last)
            self.test_ndcg_first = tf.placeholder(tf.float32)
            tf.summary.scalar('test_ndcg_first', self.test_ndcg_first)
            self.test_ndcg_last = tf.placeholder(tf.float32)
            tf.summary.scalar('test_ndcg_last', self.test_ndcg_last)
        self.merged_test_acc = tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES, 'TEST_ACC'))
        """
        *********************************************************
        Create Model Parameters (i.e., Initialize Weights).
        """
        # 初始化权重
        self.weights = self._init_weights()

        """
        *********************************************************
        Compute Graph-based Representations of all users & items via Message-Passing Mechanism of Graph Neural Networks.
        Different Convolutional Layers:
            1. ngcf: defined in 'Neural Graph Collaborative Filtering', SIGIR2019;
            2. gcn:  defined in 'Semi-Supervised Classification with Graph Convolutional Networks', ICLR2018;
            3. gcmc: defined in 'Graph Convolutional Matrix Completion', KDD2018;
        """
        
        # 根据之前代码中的定义来确定模型、算法类型，可选四种模型
        if self.alg_type in ['lightgcn']:
            self.ua_embeddings, self.ia_embeddings = self._create_lightgcn_embed()
            
        elif self.alg_type in ['ngcf']:
            self.ua_embeddings, self.ia_embeddings = self._create_ngcf_embed()

        elif self.alg_type in ['gcn']:
            self.ua_embeddings, self.ia_embeddings = self._create_gcn_embed()

        elif self.alg_type in ['gcmc']:
            self.ua_embeddings, self.ia_embeddings = self._create_gcmc_embed()

        """
        *********************************************************
        Establish the final representations for user-item pairs in batch.
        """
        # 获取嵌入向量，类似于Forward函数
        self.u_g_embeddings = tf.nn.embedding_lookup(self.ua_embeddings, self.users)
        self.pos_i_g_embeddings = tf.nn.embedding_lookup(self.ia_embeddings, self.pos_items)
        self.neg_i_g_embeddings = tf.nn.embedding_lookup(self.ia_embeddings, self.neg_items)
        self.u_g_embeddings_pre = tf.nn.embedding_lookup(self.weights['user_embedding'], self.users)
        self.pos_i_g_embeddings_pre = tf.nn.embedding_lookup(self.weights['item_embedding'], self.pos_items)
        self.neg_i_g_embeddings_pre = tf.nn.embedding_lookup(self.weights['item_embedding'], self.neg_items)

        """
        *********************************************************
        Inference for the testing phase.
        """
        # 计算内积得分，将用户向量和物品向量作内积，数值越大的两个向量越相似，反映物品更适合该用户
        self.batch_ratings = tf.matmul(self.u_g_embeddings, self.pos_i_g_embeddings, transpose_a=False, transpose_b=True)

        """
        *********************************************************
        Generate Predictions & Optimize via BPR loss.
        """
        # 定义损失函数(贝叶斯排序)和优化器(Adam)
        self.mf_loss, self.emb_loss, self.reg_loss = self.create_bpr_loss(self.u_g_embeddings,
                                                                          self.pos_i_g_embeddings,
                                                                          self.neg_i_g_embeddings)
        self.loss = self.mf_loss + self.emb_loss

        self.opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
    
    # 记录模型路径
    def create_model_str(self):
        log_dir = '/' + self.alg_type+'/layers_'+str(self.n_layers)+'/dim_'+str(self.emb_dim)
        log_dir+='/'+args.dataset+'/lr_' + str(self.lr) + '/reg_' + str(self.decay)
        return log_dir

    
    """
    LightGCN中，训练的对象其实仅是初始化的数据嵌入E0
    """
    # 初始化权重函数
    def _init_weights(self):
        all_weights = dict() 
        initializer = tf.random_normal_initializer(stddev=0.01) #tf.contrib.layers.xavier_initializer()
        # 是否使用预训练数据
        if self.pretrain_data is None:
            # 随机数据初始嵌入
            all_weights['user_embedding'] = tf.Variable(initializer([self.n_users, self.emb_dim]), name='user_embedding')
            all_weights['item_embedding'] = tf.Variable(initializer([self.n_items, self.emb_dim]), name='item_embedding')
            print('using random initialization')#print('using xavier initialization')
        else:
            # 预训练数据初始嵌入
            all_weights['user_embedding'] = tf.Variable(initial_value=self.pretrain_data['user_embed'], trainable=True,
                                                        name='user_embedding', dtype=tf.float32)
            all_weights['item_embedding'] = tf.Variable(initial_value=self.pretrain_data['item_embed'], trainable=True,
                                                        name='item_embedding', dtype=tf.float32)
            print('using pretrained initialization')
            
        self.weight_size_list = [self.emb_dim] + self.weight_size
        
        # 初始化若干矩阵，图卷积、双向网络、mlp，供上文中的四种不同模型使用（LightGCN不需要权重矩阵，因此只有三个）
        for k in range(self.n_layers):
            all_weights['W_gc_%d' %k] = tf.Variable(
                initializer([self.weight_size_list[k], self.weight_size_list[k+1]]), name='W_gc_%d' % k)
            all_weights['b_gc_%d' %k] = tf.Variable(
                initializer([1, self.weight_size_list[k+1]]), name='b_gc_%d' % k)

            all_weights['W_bi_%d' % k] = tf.Variable(
                initializer([self.weight_size_list[k], self.weight_size_list[k + 1]]), name='W_bi_%d' % k)
            all_weights['b_bi_%d' % k] = tf.Variable(
                initializer([1, self.weight_size_list[k + 1]]), name='b_bi_%d' % k)

            all_weights['W_mlp_%d' % k] = tf.Variable(
                initializer([self.weight_size_list[k], self.weight_size_list[k+1]]), name='W_mlp_%d' % k)
            all_weights['b_mlp_%d' % k] = tf.Variable(
                initializer([1, self.weight_size_list[k+1]]), name='b_mlp_%d' % k)

        return all_weights
    
    # 将原始的邻接矩阵分割成若干个子矩阵，减少资源浪费，提高训练速度
    def _split_A_hat(self, X):
        A_fold_hat = []
        
        fold_len = (self.n_users + self.n_items) // self.n_fold
        for i_fold in range(self.n_fold):
            start = i_fold * fold_len
            if i_fold == self.n_fold -1:
                end = self.n_users + self.n_items
            else:
                end = (i_fold + 1) * fold_len
            # 转为Tensorflow的稀疏张量
            A_fold_hat.append(self._convert_sp_mat_to_sp_tensor(X[start:end]))
        return A_fold_hat
    
    # 进一步加入dropout
    def _split_A_hat_node_dropout(self, X):
        A_fold_hat = []

        fold_len = (self.n_users + self.n_items) // self.n_fold
        for i_fold in range(self.n_fold):
            start = i_fold * fold_len
            if i_fold == self.n_fold -1:
                end = self.n_users + self.n_items
            else:
                end = (i_fold + 1) * fold_len

            temp = self._convert_sp_mat_to_sp_tensor(X[start:end])
            n_nonzero_temp = X[start:end].count_nonzero()
            A_fold_hat.append(self._dropout_sparse(temp, 1 - self.node_dropout[0], n_nonzero_temp))

        return A_fold_hat

    # LightGCN 的嵌入层运算
    def _create_lightgcn_embed(self):
        # 切分子矩阵
        if self.node_dropout_flag:
            A_fold_hat = self._split_A_hat_node_dropout(self.norm_adj)
        else:
            A_fold_hat = self._split_A_hat(self.norm_adj)
        
        # self.weights中的表示，将user_embedding和item_embedding拼接
        ego_embeddings = tf.concat([self.weights['user_embedding'], self.weights['item_embedding']], axis=0)
        all_embeddings = [ego_embeddings]
        
        for k in range(0, self.n_layers):

            temp_embed = []
            # 将切分的稀疏张量与密集张量相乘,其实结果就是A度×嵌入,得到更新后的E
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], ego_embeddings))
            # 再将所有向量拼接起来，拼接完成后的形状为(n_users + n_items) × emb_dim
            side_embeddings = tf.concat(temp_embed, 0)
            ego_embeddings = side_embeddings
            all_embeddings += [ego_embeddings]
        # 按照第1维度拼接
        all_embeddings=tf.stack(all_embeddings,1)
        # 嵌入均值
        all_embeddings=tf.reduce_mean(all_embeddings,axis=1,keepdims=False)
        # 拆分为用户的嵌入向量和物品的嵌入向量
        u_g_embeddings, i_g_embeddings = tf.split(all_embeddings, [self.n_users, self.n_items], 0)
        return u_g_embeddings, i_g_embeddings

    """
    def _create_ngcf_embed(self):
        if self.node_dropout_flag:
            A_fold_hat = self._split_A_hat_node_dropout(self.norm_adj)
        else:
            A_fold_hat = self._split_A_hat(self.norm_adj)

        ego_embeddings = tf.concat([self.weights['user_embedding'], self.weights['item_embedding']], axis=0)

        all_embeddings = [ego_embeddings]

        for k in range(0, self.n_layers):

            temp_embed = []
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], ego_embeddings))

            side_embeddings = tf.concat(temp_embed, 0)
            sum_embeddings = tf.nn.leaky_relu(tf.matmul(side_embeddings, self.weights['W_gc_%d' % k]) + self.weights['b_gc_%d' % k])



            # bi messages of neighbors.
            bi_embeddings = tf.multiply(ego_embeddings, side_embeddings)
            # transformed bi messages of neighbors.
            bi_embeddings = tf.nn.leaky_relu(tf.matmul(bi_embeddings, self.weights['W_bi_%d' % k]) + self.weights['b_bi_%d' % k])
            # non-linear activation.
            ego_embeddings = sum_embeddings + bi_embeddings

            # message dropout.
            # ego_embeddings = tf.nn.dropout(ego_embeddings, 1 - self.mess_dropout[k])

            # normalize the distribution of embeddings.
            norm_embeddings = tf.nn.l2_normalize(ego_embeddings, axis=1)

            all_embeddings += [norm_embeddings]

        all_embeddings = tf.concat(all_embeddings, 1)
        u_g_embeddings, i_g_embeddings = tf.split(all_embeddings, [self.n_users, self.n_items], 0)
        return u_g_embeddings, i_g_embeddings
    
    
    def _create_gcn_embed(self):
        A_fold_hat = self._split_A_hat(self.norm_adj)
        embeddings = tf.concat([self.weights['user_embedding'], self.weights['item_embedding']], axis=0)


        all_embeddings = [embeddings]

        for k in range(0, self.n_layers):
            temp_embed = []
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], embeddings))

            embeddings = tf.concat(temp_embed, 0)
            embeddings = tf.nn.leaky_relu(tf.matmul(embeddings, self.weights['W_gc_%d' %k]) + self.weights['b_gc_%d' %k])
            # embeddings = tf.nn.dropout(embeddings, 1 - self.mess_dropout[k])

            all_embeddings += [embeddings]

        all_embeddings = tf.concat(all_embeddings, 1)
        u_g_embeddings, i_g_embeddings = tf.split(all_embeddings, [self.n_users, self.n_items], 0)
        return u_g_embeddings, i_g_embeddings
    
    def _create_gcmc_embed(self):
        A_fold_hat = self._split_A_hat(self.norm_adj)

        embeddings = tf.concat([self.weights['user_embedding'], self.weights['item_embedding']], axis=0)

        all_embeddings = []

        for k in range(0, self.n_layers):
            temp_embed = []
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], embeddings))
            embeddings = tf.concat(temp_embed, 0)
            # convolutional layer.
            embeddings = tf.nn.leaky_relu(tf.matmul(embeddings, self.weights['W_gc_%d' % k]) + self.weights['b_gc_%d' % k])
            # dense layer.
            mlp_embeddings = tf.matmul(embeddings, self.weights['W_mlp_%d' %k]) + self.weights['b_mlp_%d' %k]
            # mlp_embeddings = tf.nn.dropout(mlp_embeddings, 1 - self.mess_dropout[k])

            all_embeddings += [mlp_embeddings]
        all_embeddings = tf.concat(all_embeddings, 1)

        u_g_embeddings, i_g_embeddings = tf.split(all_embeddings, [self.n_users, self.n_items], 0)
        return u_g_embeddings, i_g_embeddings
    """
    # 损失函数 贝叶斯个性化排序
    def create_bpr_loss(self, users, pos_items, neg_items):
        # 获取正样本喜好度和负样本喜好度
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1)
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1)
        
        regularizer = tf.nn.l2_loss(self.u_g_embeddings_pre) + tf.nn.l2_loss(
                self.pos_i_g_embeddings_pre) + tf.nn.l2_loss(self.neg_i_g_embeddings_pre)
        regularizer = regularizer / self.batch_size
        
        # softplus(x) = log(1 + exp(x))
        mf_loss = tf.reduce_mean(tf.nn.softplus(-(pos_scores - neg_scores)))
        

        emb_loss = self.decay * regularizer

        reg_loss = tf.constant(0.0, tf.float32, [1])

        return mf_loss, emb_loss, reg_loss
    
    # 将稀疏矩阵转为 TensorFlow 中的稀疏张量，以便在TensorFlow 中进行计算
    """
    coo对象：稀疏矩阵，每一条的格式为 （行 列 非零值）
    indices：仅取出非零值的位置
    SparseTensor：稀疏张量，存储为：（indices values dense_shape）
    数据格式例：
    indices = [[0, 1], [1, 0], [2, 2], [2, 3]]  # 4个非零元素的位置
    values = [1, 2, 3, 4]  # 4个非零元素的值
    dense_shape = [3, 4]  # 张量的形状为3行4列
    """
    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        indices = np.mat([coo.row, coo.col]).transpose()
        return tf.SparseTensor(indices, coo.data, coo.shape)
    """
    def _dropout_sparse(self, X, keep_prob, n_nonzero_elems):
        
        # Dropout for sparse tensors.
        
        noise_shape = [n_nonzero_elems]
        random_tensor = keep_prob
        random_tensor += tf.random_uniform(noise_shape)
        dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
        pre_out = tf.sparse_retain(X, dropout_mask)

        return pre_out * tf.div(1., keep_prob)
    """

In [5]:
# 读取预训练的数据（前提要有预训练的数据）
def load_pretrained_data():
    pretrain_path = '%spretrain/%s/%s.npz' % (args.proj_path, args.dataset, 'embedding')
    try:
        pretrain_data = np.load(pretrain_path)
        print('load the pretrained embeddings.')
    except Exception:
        pretrain_data = None
    return pretrain_data

In [6]:
# CPU并行处理
# parallelized sampling on CPU 
class sample_thread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        with tf.device(cpus[0]):
            self.data = data_generator.sample()

In [7]:
class sample_thread_test(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        with tf.device(cpus[0]):
            self.data = data_generator.sample_test()

In [8]:
# GPU并行处理
# training on GPU
class train_thread(threading.Thread):
    def __init__(self,model, sess, sample):
        threading.Thread.__init__(self)
        self.model = model
        self.sess = sess
        self.sample = sample
    def run(self):
#         with tf.device(gpus[1]):
#             with tf.degice('/CPU:0'):
        users, pos_items, neg_items = self.sample.data
        self.data = sess.run([self.model.opt, self.model.loss, self.model.mf_loss, self.model.emb_loss, self.model.reg_loss],
                                feed_dict={model.users: users, model.pos_items: pos_items,
                                            model.node_dropout: eval(args.node_dropout),
                                            model.mess_dropout: eval(args.mess_dropout),
                                            model.neg_items: neg_items})


In [9]:
class train_thread_test(threading.Thread):
    def __init__(self,model, sess, sample):
        threading.Thread.__init__(self)
        self.model = model
        self.sess = sess
        self.sample = sample
    def run(self):
#         with tf.device(gpus[1]):
        users, pos_items, neg_items = self.sample.data
        self.data = sess.run([self.model.loss, self.model.mf_loss, self.model.emb_loss],
                            feed_dict={model.users: users, model.pos_items: pos_items,
                                    model.neg_items: neg_items,
                                    model.node_dropout: eval(args.node_dropout),
                                    model.mess_dropout: eval(args.mess_dropout)})   

# Main

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu_id)
f0 = time()

config = dict()
config['n_users'] = data_generator.n_users
config['n_items'] = data_generator.n_items

In [11]:
"""
*********************************************************
Generate the Laplacian matrix, where each entry defines the decay factor (e.g., p_ui) between two connected nodes.
"""
# 获取邻接矩阵
plain_adj, norm_adj, mean_adj,pre_adj = data_generator.get_adj_mat()
if args.adj_type == 'plain':
    config['norm_adj'] = plain_adj
    print('use the plain adjacency matrix')
elif args.adj_type == 'norm':
    config['norm_adj'] = norm_adj
    print('use the normalized adjacency matrix')
elif args.adj_type == 'gcmc':
    config['norm_adj'] = mean_adj
    print('use the gcmc adjacency matrix')
elif args.adj_type=='pre':
    config['norm_adj']=pre_adj
    print('use the pre adjcency matrix')
else:
    config['norm_adj'] = mean_adj + sp.eye(mean_adj.shape[0])
    print('use the mean adjacency matrix')
t0 = time()
if args.pretrain == -1:
    pretrain_data = load_pretrained_data()
else:
    pretrain_data = None
model = LightGCN(data_config=config, pretrain_data=pretrain_data)

already load adj matrix (70839, 70839) 0.15360474586486816
use the pre adjcency matrix
using random initialization


In [12]:
# 保存模型参数
"""
*********************************************************
Save the model parameters.
"""
saver = tf.train.Saver()

if args.save_flag == 1:
    layer = '-'.join([str(l) for l in eval(args.layer_size)])
    weights_save_path = '%sweights/%s/%s/%s/l%s_r%s' % (args.weights_path, args.dataset, model.model_type, layer,
                                                        str(args.lr), '-'.join([str(r) for r in eval(args.regs)]))
    ensureDir(weights_save_path)
    save_saver = tf.train.Saver(max_to_keep=1)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
# 读取模型参数
"""
*********************************************************
Reload the pretrained model parameters.
"""
if args.pretrain == 1:
    layer = '-'.join([str(l) for l in eval(args.layer_size)])

    pretrain_path = '%sweights/%s/%s/%s/l%s_r%s' % (args.weights_path, args.dataset, model.model_type, layer,
                                                    str(args.lr), '-'.join([str(r) for r in eval(args.regs)]))


    ckpt = tf.train.get_checkpoint_state(os.path.dirname(pretrain_path + '/checkpoint'))
    if ckpt and ckpt.model_checkpoint_path:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, ckpt.model_checkpoint_path)
        print('load the pretrained model parameters from: ', pretrain_path)

        # *********************************************************
        # get the performance from pretrained model.
        if args.report != 1:
            users_to_test = list(data_generator.test_set.keys())
            ret = test(sess, model, users_to_test, drop_flag=True)
            cur_best_pre_0 = ret['recall'][0]

            pretrain_ret = 'pretrained model recall=[%s], precision=[%s], '\
                           'ndcg=[%s]' % \
                           (', '.join(['%.5f' % r for r in ret['recall']]),
                            ', '.join(['%.5f' % r for r in ret['precision']]),
                            ', '.join(['%.5f' % r for r in ret['ndcg']]))
            print(pretrain_ret)
    else:
        sess.run(tf.global_variables_initializer())
        cur_best_pre_0 = 0.
        print('without pretraining.')

else:
    sess.run(tf.global_variables_initializer())
    cur_best_pre_0 = 0.
    print('without pretraining.')

without pretraining.


In [ ]:
"""
*********************************************************
Get the performance w.r.t. different sparsity levels.
"""
if args.report == 1:
    assert args.test_flag == 'full'
    users_to_test_list, split_state = data_generator.get_sparsity_split()
    users_to_test_list.append(list(data_generator.test_set.keys()))
    split_state.append('all')

    report_path = '%sreport/%s/%s.result' % (args.proj_path, args.dataset, model.model_type)
    ensureDir(report_path)
    f = open(report_path, 'w')
    f.write(
        'embed_size=%d, lr=%.4f, layer_size=%s, keep_prob=%s, regs=%s, loss_type=%s, adj_type=%s\n'
        % (args.embed_size, args.lr, args.layer_size, args.keep_prob, args.regs, args.loss_type, args.adj_type))

    for i, users_to_test in enumerate(users_to_test_list):
        ret = test(sess, model, users_to_test, drop_flag=True)

        final_perf = "recall=[%s], precision=[%s], ndcg=[%s]" % \
                     (', '.join(['%.5f' % r for r in ret['recall']]),
                      ', '.join(['%.5f' % r for r in ret['precision']]),
                      ', '.join(['%.5f' % r for r in ret['ndcg']]))

        f.write('\t%s\n\t%s\n' % (split_state[i], final_perf))
    f.close()
    exit()

In [ ]:
"""
*********************************************************
Train.
"""
tensorboard_model_path = 'tensorboard/'
if not os.path.exists(tensorboard_model_path):
    os.makedirs(tensorboard_model_path)
run_time = 1
while (True):
    if os.path.exists(tensorboard_model_path + model.log_dir +'/run_' + str(run_time)):
        run_time += 1
    else:
        break
train_writer = tf.summary.FileWriter(tensorboard_model_path +model.log_dir+ '/run_' + str(run_time), sess.graph)


loss_loger, pre_loger, rec_loger, ndcg_loger, hit_loger = [], [], [], [], []
stopping_step = 0
should_stop = False


for epoch in range(1, args.epoch + 1):
    print("Epoch:",epoch)
    t1 = time()
    loss, mf_loss, emb_loss, reg_loss = 0., 0., 0., 0.
    n_batch = data_generator.n_train // args.batch_size + 1
    loss_test,mf_loss_test,emb_loss_test,reg_loss_test=0.,0.,0.,0.
    '''
    *********************************************************
    parallelized sampling
    '''
    sample_last = sample_thread()
    sample_last.start()
    sample_last.join()
    for idx in range(n_batch):
        train_cur = train_thread(model, sess, sample_last)
        sample_next = sample_thread()

        train_cur.start()
        sample_next.start()

        sample_next.join()
        train_cur.join()
        print('r')
        users, pos_items, neg_items = sample_last.data
        _, batch_loss, batch_mf_loss, batch_emb_loss, batch_reg_loss = train_cur.data
        sample_last = sample_next

        loss += batch_loss/n_batch
        mf_loss += batch_mf_loss/n_batch
        emb_loss += batch_emb_loss/n_batch

    summary_train_loss= sess.run(model.merged_train_loss,
                                  feed_dict={model.train_loss: loss, model.train_mf_loss: mf_loss,
                                             model.train_emb_loss: emb_loss, model.train_reg_loss: reg_loss})
    train_writer.add_summary(summary_train_loss, epoch)
    if np.isnan(loss) == True:
        print('ERROR: loss is nan.')
        sys.exit()

    if (epoch % 20) != 0:
        if args.verbose > 0 and epoch % args.verbose == 0:
            perf_str = 'Epoch %d [%.1fs]: train==[%.5f=%.5f + %.5f]' % (
                epoch, time() - t1, loss, mf_loss, emb_loss)
            print(perf_str)
        continue
        
    users_to_test = list(data_generator.train_items.keys())
    ret = test(sess, model, users_to_test ,drop_flag=True,train_set_flag=1)
    perf_str = 'Epoch %d: train==[%.5f=%.5f + %.5f + %.5f], recall=[%s], precision=[%s], ndcg=[%s]' % \
               (epoch, loss, mf_loss, emb_loss, reg_loss, 
                ', '.join(['%.5f' % r for r in ret['recall']]),
                ', '.join(['%.5f' % r for r in ret['precision']]),
                ', '.join(['%.5f' % r for r in ret['ndcg']]))
    print(perf_str)
    summary_train_acc = sess.run(model.merged_train_acc, feed_dict={model.train_rec_first: ret['recall'][0],
                                                                    model.train_rec_last: ret['recall'][-1],
                                                                    model.train_ndcg_first: ret['ndcg'][0],
                                                                    model.train_ndcg_last: ret['ndcg'][-1]})
    train_writer.add_summary(summary_train_acc, epoch // 20)

    '''
    *********************************************************
    parallelized sampling
    '''
    sample_last= sample_thread_test()
    sample_last.start()
    sample_last.join()
    for idx in range(n_batch):
        train_cur = train_thread_test(model, sess, sample_last)
        sample_next = sample_thread_test()

        train_cur.start()
        sample_next.start()

        sample_next.join()
        train_cur.join()

        users, pos_items, neg_items = sample_last.data
        batch_loss_test, batch_mf_loss_test, batch_emb_loss_test = train_cur.data
        sample_last = sample_next

        loss_test += batch_loss_test / n_batch
        mf_loss_test += batch_mf_loss_test / n_batch
        emb_loss_test += batch_emb_loss_test / n_batch

    summary_test_loss = sess.run(model.merged_test_loss,
                                 feed_dict={model.test_loss: loss_test, model.test_mf_loss: mf_loss_test,
                                            model.test_emb_loss: emb_loss_test, model.test_reg_loss: reg_loss_test})
    train_writer.add_summary(summary_test_loss, epoch // 20)
    t2 = time()
    users_to_test = list(data_generator.test_set.keys())
    ret = test(sess, model, users_to_test, drop_flag=True)
    summary_test_acc = sess.run(model.merged_test_acc,
                                feed_dict={model.test_rec_first: ret['recall'][0], model.test_rec_last: ret['recall'][-1],
                                           model.test_ndcg_first: ret['ndcg'][0], model.test_ndcg_last: ret['ndcg'][-1]})
    train_writer.add_summary(summary_test_acc, epoch // 20)


    t3 = time()

    loss_loger.append(loss)
    rec_loger.append(ret['recall'])
    pre_loger.append(ret['precision'])
    ndcg_loger.append(ret['ndcg'])

    if args.verbose > 0:
        perf_str = 'Epoch %d [%.1fs + %.1fs]: test==[%.5f=%.5f + %.5f + %.5f], recall=[%s], ' \
                   'precision=[%s], ndcg=[%s]' % \
                   (epoch, t2 - t1, t3 - t2, loss_test, mf_loss_test, emb_loss_test, reg_loss_test, 
                    ', '.join(['%.5f' % r for r in ret['recall']]),
                    ', '.join(['%.5f' % r for r in ret['precision']]),
                    ', '.join(['%.5f' % r for r in ret['ndcg']]))
        print(perf_str)

    cur_best_pre_0, stopping_step, should_stop = early_stopping(ret['recall'][0], cur_best_pre_0,
                                                                stopping_step, expected_order='acc', flag_step=5)

    # *********************************************************
    # early stopping when cur_best_pre_0 is decreasing for ten successive steps.
    if should_stop == True:
        break

    # *********************************************************
    # save the user & item embeddings for pretraining.
    if ret['recall'][0] == cur_best_pre_0 and args.save_flag == 1:
        save_saver.save(sess, weights_save_path + '/weights', global_step=epoch)
        print('save the weights in path: ', weights_save_path)

In [ ]:
recs = np.array(rec_loger)
pres = np.array(pre_loger)
ndcgs = np.array(ndcg_loger)

best_rec_0 = max(recs[:, 0])
idx = list(recs[:, 0]).index(best_rec_0)

final_perf = "Best Iter=[%d]@[%.1f]\trecall=[%s], precision=[%s], ndcg=[%s]" % \
             (idx, time() - t0, '\t'.join(['%.5f' % r for r in recs[idx]]),
              '\t'.join(['%.5f' % r for r in pres[idx]]),
              '\t'.join(['%.5f' % r for r in ndcgs[idx]]))
print(final_perf)

save_path = '%soutput/%s/%s.result' % (args.proj_path, args.dataset, model.model_type)
ensureDir(save_path)
f = open(save_path, 'a')

f.write(
    'embed_size=%d, lr=%.4f, layer_size=%s, node_dropout=%s, mess_dropout=%s, regs=%s, adj_type=%s\n\t%s\n'
    % (args.embed_size, args.lr, args.layer_size, args.node_dropout, args.mess_dropout, args.regs,
       args.adj_type, final_perf))
f.close()